## this is title


In [ ]:
import sys
import os

PROJECT_ROOT = os.getcwd()  # already inside backend
sys.path.insert(0, PROJECT_ROOT)

print("Using project root:", PROJECT_ROOT)


Using project root: /Users/mahimaadvilkar/Desktop/ALL ABOUT AI/HealthOps/backend/src/services


In [11]:
from src.services.referral_intelligence_agent import ReferralIntelligenceAgent
from src.services.referral_reasoning_agent import ReferralReasoningAgent

In [12]:
intel = ReferralIntelligenceAgent()
reason = ReferralReasoningAgent()

print("Agents loaded successfully ✅")


Agents loaded successfully ✅


In [13]:
import sys
import os
from datetime import date

# Notebook is already inside backend/, so use cwd
PROJECT_ROOT = os.getcwd()
sys.path.insert(0, PROJECT_ROOT)

from src.services.referral_intelligence_agent import ReferralIntelligenceAgent
from src.services.referral_reasoning_agent import ReferralReasoningAgent

print("Setup complete ✅")


Setup complete ✅


In [14]:
referral = {
    "referral_id": "REF-1013",
    "insurance_active": "Y",
    "auth_required": "Y",
    "auth_status": "APPROVED",
    "auth_start_date": date(2025, 12, 22),
    "auth_end_date": date(2026, 1, 12),
    "docs_complete": "Y",
    "schedule_status": "NOT_SCHEDULED",
    "units_delivered_to_date": 0,
    "ready_to_bill": "N"
}

print("Referral loaded:", referral["referral_id"])


Referral loaded: REF-1013


In [15]:
# Initialize agents
intel_agent = ReferralIntelligenceAgent()
reasoning_agent = ReferralReasoningAgent()

# Step 1: Policy / Intelligence decision
decision = intel_agent.evaluate(referral)

print("=== INTELLIGENCE AGENT OUTPUT ===")
print(decision)

# Step 2: Reasoning / Explanation
explanation = reasoning_agent.explain(referral, decision)

print("\n=== REASONING AGENT OUTPUT ===")
print(explanation)


=== INTELLIGENCE AGENT OUTPUT ===
{'agent_segment': 'ORANGE', 'agent_next_action': 'ESCALATE', 'agent_rationale': 'Authorization expiring soon and not scheduled; escalate immediately.'}

=== REASONING AGENT OUTPUT ===
{'llm_explanation': 'This referral is currently classified as ORANGE.\n        The recommended action is ESCALATE.\n        Reason: Authorization expiring soon and not scheduled; escalate immediately.', 'risk_summary': 'Time-sensitive risk; immediate action required.', 'ops_recommendation': 'Escalate to supervisor and prioritize scheduling within 24 hours.'}


In [16]:
 # Simulated backlog (3–5 referrals is enough for demo)
backlog = [
    {
        "referral_id": "REF-1003",
        "insurance_active": "Y",
        "auth_required": "Y",
        "auth_status": "APPROVED",
        "auth_end_date": date(2026, 1, 10),
        "docs_complete": "Y",
        "schedule_status": "NOT_SCHEDULED",
        "ready_to_bill": "N"
    },
    {
        "referral_id": "REF-1007",
        "insurance_active": "Y",
        "auth_required": "Y",
        "auth_status": "APPROVED",
        "auth_end_date": date(2026, 1, 15),
        "docs_complete": "N",
        "schedule_status": "NOT_SCHEDULED",
        "ready_to_bill": "N"
    },
    {
        "referral_id": "REF-1010",
        "insurance_active": "N",
        "auth_required": "Y",
        "auth_status": "PENDING",
        "docs_complete": "Y",
        "schedule_status": "NOT_SCHEDULED",
        "ready_to_bill": "N"
    }
]

action_queue = []

for referral in backlog:
    decision = intel_agent.evaluate(referral)
    reasoning = reasoning_agent.explain(referral, decision)

    action_queue.append({
        "referral_id": referral["referral_id"],
        "segment": decision["agent_segment"],
        "next_action": decision["agent_next_action"],
        "risk_summary": reasoning["risk_summary"],
        "ops_recommendation": reasoning["ops_recommendation"]
    })

# Sort by urgency (RED > ORANGE > YELLOW > GREEN)
priority_order = {"RED": 0, "ORANGE": 1, "YELLOW": 2, "GREEN": 3}
action_queue = sorted(
    action_queue,
    key=lambda x: priority_order[x["segment"]]
)

action_queue


[{'referral_id': 'REF-1010',
  'segment': 'RED',
  'next_action': 'HOLD',
  'risk_summary': 'High risk of revenue loss or compliance failure.',
  'ops_recommendation': 'Pause workflow until blocking issue resolved.'},
 {'referral_id': 'REF-1003',
  'segment': 'ORANGE',
  'next_action': 'ESCALATE',
  'risk_summary': 'Time-sensitive risk; immediate action required.',
  'ops_recommendation': 'Escalate to supervisor and prioritize scheduling within 24 hours.'},
 {'referral_id': 'REF-1007',
  'segment': 'GREEN',
  'next_action': 'NO_ACTION',
  'risk_summary': 'Low operational risk.',
  'ops_recommendation': 'Monitor referral.'}]